In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
!pip install openpyxl
!pip install XlsxWriter
!pip install xlrd
!pip install python-dateutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 3.1 MB/s eta 0:00:00


In [3]:
# pd.set_option('display.max_rows', None)

# input_root_path = "../data"
input_root_path = "../input/travel-time"
ats_path = "../input/atstraffic"
weather_path = "../input/weatherstations"
counter_path = "../input/counterstations"
# output_root_path = "../data"
output_root_path = "../working"


In [4]:
def load_csv_to_df(files, filename=None, is_excel=False):
    if is_excel:
        df = pd.concat((pd.read_excel(f) for f in files), ignore_index=True)
    else:
        df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)
    
    if filename:
        df.to_csv(os.path.join(output_root_path, filename), header = True, index=True)
    return df

# Pre-processing

In [5]:
# Load data 

tasks_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive/Travel Time Task/TravelTimeTaskArchive*.csv"))
df_tasks = load_csv_to_df(tasks_joined_files)
# df_tasks = load_csv_to_df(tasks_joined_files, filename="all_TravelTimeTaskArchive.csv")

summary_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive/Travel Time Summary/TravelTimeSummaryTaskArchive*.csv"))
# df_summary = load_csv_to_df(summary_joined_files)

df_tasks

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
1,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
2,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
3,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
4,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
...,...,...,...,...,...,...,...,...
13438804,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438805,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438806,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438807,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06


## Find Unique Assets


In [6]:
def get_unique_assets(filename=None):
    df_asset = pd.concat([df_tasks[['AssetNumber', 'AssetIdentifier']], df_summary[['AssetNumber', 'AssetIdentifier']]], axis=0) 
    df_asset_unique = df_asset.drop_duplicates()
    
    if filename:
        df_asset_unique.to_csv(os.path.join(output_root_path, filename), header = True, index=True)

    return df_asset_unique

In [7]:
# df_asset_unique = get_unique_assets()
# df_asset_unique = get_unique_assets(filename='unique_assets.csv')
# df_asset_unique

# Cleaning

## Get Traffic Times

In [8]:
# Filter out entries with zero UnderConstructionTime or BaselineTime
df_valid = df_tasks[(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].rename_axis('AllTasksIndex').reset_index()
df_valid = df_valid.rename_axis('ValidTasksIndex')
# df_valid.index.name = 'ValidTasksIndex'
df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
ValidTasksIndex,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,5/23/2021 4:55:09 PM,95.90,95.37,0.53,OK,146.94
1,1000001,59711435013,Spread 7 - NB,5/23/2021 4:55:09 PM,21.63,23.88,0.00,OK,33.60
2,1000002,14058367507,Spread 6 - NB,5/23/2021 4:55:09 PM,40.63,41.88,0.00,OK,72.16
3,1000003,50253718586,Spread 5A - SB,5/23/2021 4:55:10 PM,123.68,115.63,8.05,OK,185.13
4,1000004,20828851697,Spread 5A - NB,5/23/2021 4:55:10 PM,106.97,111.38,0.00,OK,185.21
...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.98
3809837,12938805,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.71
3809838,12938806,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.35


## Get Assets

In [9]:
# df_assets_all_active = pd.read_excel(os.path.join(ats_path, 'AllWorkZones.xlsx'), sheet_name = ['HaveBeenActivated'])['HaveBeenActivated']
# df_assets_all_active

In [10]:
df_assets_recom = pd.read_excel(os.path.join(ats_path, 'Recommended Assets.xlsx'))
df_assets_recom

,Asset Number,Segment Name,Segment Length,Under Construction,Baseline,Direct of Travel,Spread,Traffic Impact,Details of Right of Way to work within or close,Description of work,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage
0,62134790681,AP 3-B-12-A-1,2.746,1.67,1.72,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
1,10897197432,AP 3-B-12-A-1,2.746,1.68,1.73,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
2,12768050215,AP 3-B-13-A-1,2.970,1.77,1.92,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
3,56678162306,AP 3-B-13-A-1,2.970,1.80,1.92,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NaN,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of an Access Point along Spread 3-4A o...,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,0.00,SB,Spread 5A,High,SMJV is requesting approval to work within the...,This application is for traffic control at the...,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,0.00,SB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,0.00,NB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.73,2.58,SB,Reactivation,High,I/W request approval to work within the Minist...,"The eastbound passing lane will be closed, the...",Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area


In [11]:
# # Remove No Cell Coverage Area entries
# df_assets_recom_no_cell = df_assets_recom[df_assets_recom['Cell Coverage'] != 'No Cell Coverage Area']
# df_assets_recom_no_cell

In [12]:
# df_assets_all_active.columns.difference(df_assets_recom.columns)

## Recalculate Delay Time & State

In [13]:
# Recalculate DelayTime 
df_valid['CalculatedDelayTime'] = (df_valid.UnderConstructionTime - df_valid.BaselineTime).clip(lower=0).round(4)
df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime
ValidTasksIndex,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,5/23/2021 4:55:09 PM,95.90,95.37,0.53,OK,146.94,0.53
1,1000001,59711435013,Spread 7 - NB,5/23/2021 4:55:09 PM,21.63,23.88,0.00,OK,33.60,0.00
2,1000002,14058367507,Spread 6 - NB,5/23/2021 4:55:09 PM,40.63,41.88,0.00,OK,72.16,0.00
3,1000003,50253718586,Spread 5A - SB,5/23/2021 4:55:10 PM,123.68,115.63,8.05,OK,185.13,8.05
4,1000004,20828851697,Spread 5A - NB,5/23/2021 4:55:10 PM,106.97,111.38,0.00,OK,185.21,0.00
...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.98,0.00
3809837,12938805,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.71,0.00
3809838,12938806,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.35,0.00


In [14]:
# # Recalculate State
# df_valid['CalculatedState'] = np.where(df_valid['CalculatedDelayTime'] < DELAY_THRESH, 'OK', 'DELAY')
# df_valid

## Extract New Columns

### From 'CreatedDate'

In [15]:
# DayType:
#     1: Monday (0) & Friday (4)
#     2: OtherWeekday (1,2,3)
#     3: Weekend (5,6)
dict_day_type = {0:1, 1:2, 2:2, 3:2, 4:1, 5:3, 6:3}

# HourType: 
#     1: 6-10
#     2: 10-15
#     3: 15-18
#     4: 18-22
#     5: 22-6
dict_hour_type = {6:1, 7:1, 8:1,9:1, 
                  10:2,11:2,12:2,13:2,14:2,
                  15:3,16:3,17:3,
                  18:4,19:4,20:4,21:4,
                  22:5,23:5,0:5,1:5,2:5,3:5,4:5,5:5}

In [16]:
df_valid['CreatedDate'] = pd.to_datetime(df_valid['CreatedDate'])

# Return the day of the week as an integer, where Monday is 1 and Sunday is 8.
df_valid['CreatedDate:Weekday'] = df_valid['CreatedDate'].dt.weekday + 1
# df_valid['CreatedDate:DayName'] = df_valid['CreatedDate'].dt.day_name()

# df_valid['CreatedDate:WeekOfYear'] = df_valid['CreatedDate'].dt.week_of_year
# df_valid['CreatedDate:DayOfYear'] = df_valid['CreatedDate'].dt.day_of_year

df_valid['CreatedDate:Year'] = df_valid['CreatedDate'].dt.year
df_valid['CreatedDate:Month'] = df_valid['CreatedDate'].dt.month
# # Between 1 and the number of days in the given month of the given year.
# df_valid['CreatedDate:Day'] = df_valid['CreatedDate'].dt.day
df_valid['CreatedDate:Hour'] = df_valid['CreatedDate'].dt.hour
# df_valid['CreatedDate:Minute'] = df_valid['CreatedDate'].dt.minute

# Seasons:
# 1: Winter – December, January and February.
# 2: Spring – March, April and May.
# 3: Summer – June, July and August.
# 4: Autumn – September, October and November.
df_valid['CreatedDate:Season'] = df_valid['CreatedDate'].dt.month%12 // 3 + 1

df_valid['CreatedDate:DayType'] = df_valid['CreatedDate'].dt.weekday.map(dict_day_type)

df_valid['CreatedDate:HourType'] = df_valid['CreatedDate'].dt.hour.map(dict_hour_type)


df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType
ValidTasksIndex,,,,,,,,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,2021-05-23 16:55:09,95.90,95.37,0.53,OK,146.94,0.53,7,2021,5,16,2,3,3
1,1000001,59711435013,Spread 7 - NB,2021-05-23 16:55:09,21.63,23.88,0.00,OK,33.60,0.00,7,2021,5,16,2,3,3
2,1000002,14058367507,Spread 6 - NB,2021-05-23 16:55:09,40.63,41.88,0.00,OK,72.16,0.00,7,2021,5,16,2,3,3
3,1000003,50253718586,Spread 5A - SB,2021-05-23 16:55:10,123.68,115.63,8.05,OK,185.13,8.05,7,2021,5,16,2,3,3
4,1000004,20828851697,Spread 5A - NB,2021-05-23 16:55:10,106.97,111.38,0.00,OK,185.21,0.00,7,2021,5,16,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,2021-04-17 18:00:11,21.77,23.60,0.00,OK,32.98,0.00,6,2021,4,18,2,3,4
3809837,12938805,75946582969,Spread 4B - NB,2021-04-17 18:00:12,43.37,45.35,0.00,OK,70.71,0.00,6,2021,4,18,2,3,4
3809838,12938806,43132175891,Spread 5B - SB,2021-04-17 18:00:12,53.48,53.73,0.00,OK,87.35,0.00,6,2021,4,18,2,3,4


### From DelayTime

In [17]:
DELAY_MIN_THRESH = 20
DELAY_MAX_THRESH = 4 * DELAY_MIN_THRESH


In [18]:
# if cond1:
#     exp1
# elif cond2:
#     exp2
# else:
#     exp3
# np.where(cond1, exp1, np.where(cond2, exp2, ...))

df_valid['DelayLevel'] = (
    np.where(df_valid['DelayTime'] == 0.0, 1,
             np.where(df_valid['DelayTime'] > DELAY_MAX_THRESH, 4,
                      np.where(df_valid['DelayTime'] > DELAY_MIN_THRESH, 3, 2))))

print("DELAY_MIN_THRESH =", DELAY_MIN_THRESH)
print("DELAY_MAX_THRESH =", DELAY_MAX_THRESH)

# df_valid.loc[df_valid['DelayLevel'] == 3]    
df_valid

DELAY_MIN_THRESH = 20
DELAY_MAX_THRESH = 80


,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
ValidTasksIndex,,,,,,,,,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,2021-05-23 16:55:09,95.90,95.37,0.53,OK,146.94,0.53,7,2021,5,16,2,3,3,2
1,1000001,59711435013,Spread 7 - NB,2021-05-23 16:55:09,21.63,23.88,0.00,OK,33.60,0.00,7,2021,5,16,2,3,3,1
2,1000002,14058367507,Spread 6 - NB,2021-05-23 16:55:09,40.63,41.88,0.00,OK,72.16,0.00,7,2021,5,16,2,3,3,1
3,1000003,50253718586,Spread 5A - SB,2021-05-23 16:55:10,123.68,115.63,8.05,OK,185.13,8.05,7,2021,5,16,2,3,3,2
4,1000004,20828851697,Spread 5A - NB,2021-05-23 16:55:10,106.97,111.38,0.00,OK,185.21,0.00,7,2021,5,16,2,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,2021-04-17 18:00:11,21.77,23.60,0.00,OK,32.98,0.00,6,2021,4,18,2,3,4,1
3809837,12938805,75946582969,Spread 4B - NB,2021-04-17 18:00:12,43.37,45.35,0.00,OK,70.71,0.00,6,2021,4,18,2,3,4,1
3809838,12938806,43132175891,Spread 5B - SB,2021-04-17 18:00:12,53.48,53.73,0.00,OK,87.35,0.00,6,2021,4,18,2,3,4,1


In [19]:
# df_valid.to_csv(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.csv'), header = True, index=True)

In [20]:
# df_valid.to_excel(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.xlsx'), header = True, index=True)

## Match work zone datasets
For all the workzones under the "have been activated" tab, creates a separate spreadsheet for each workzone that contains all the travel time information that you can find in the "Update Travel Time Archive_All_Data" spreadsheet. So for each workzone we would have one spreadsheet with all the travel time information.

In [21]:
def find_workzone_tt_data(df_assets, file_name=None):
    df_tt_workzone = pd.DataFrame()
    
    for i, location_id in enumerate(df_assets['Asset Number']):
        df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
        df_tt_workzone = df_tt_workzone.append(df_intersect)
        
        if file_name:
            with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
                df_intersect.to_excel(writer, sheet_name=str(location_id))
            
#         if (i+1) % 50 == 0:
#             print(f'{round((i+1)/len(df_assets)*100, 2)} %')
            
    return df_tt_workzone
         

In [22]:
df_tt_workzone = find_workzone_tt_data(df_assets_recom)
df_tt_workzone

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
ValidTasksIndex,,,,,,,,,,,,,,,,,,
87153,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,7,2021,5,20,2,3,4,2
87203,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,7,2021,5,20,2,3,4,2
87253,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,7,2021,5,20,2,3,4,2
87303,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,7,2021,5,20,2,3,4,1
87353,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,7,2021,5,20,2,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452149,12552139,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:40:11,1.00,1.65,0.00,OK,2.73,0.00,4,2021,3,3,2,2,5,1
3452228,12552231,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:45:11,1.00,1.65,0.00,OK,2.73,0.00,4,2021,3,3,2,2,5,1
3452301,12552314,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:50:12,1.00,1.65,0.00,OK,2.73,0.00,4,2021,3,3,2,2,5,1


In [23]:
# find_data_workzone(df_assets_all_active, file_name='TravelTimesPerWorkZone_AllActive.xlsx')
# find_data_workzone(df_assets_recom, file_name=''TravelTimesPerWorkZone_Recommended.xlsx')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_AllActive.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_all_active['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_all_active)*100, 2)} %')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_Recommended.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_recom['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_recom)*100, 2)} %')


# Get Weather Data

In [24]:
df_matched_ids = pd.read_csv(os.path.join(ats_path, 'AssetNumberStationID.csv'),index_col=0)
dict_matched_ids = df_matched_ids.set_index('Asset Number')['Station ID'].to_dict()
df_matched_ids

,Asset Number,Station ID
0,23023696729,706
1,98128214373,706
2,73174024548,706
3,40105790845,706
4,12738620179,706
...,...,...
619,55858030442,794
620,45364364608,794
621,40786939101,794
622,34684428742,49408


In [25]:
dict_weather = {}

for station_id in df_matched_ids['Station ID'].unique():
    weather_joined_files = glob.glob(os.path.join(weather_path, f'{station_id}/*.csv'))
    df_weather = load_csv_to_df(weather_joined_files)
    
    dict_weather[station_id] = df_weather
    dict_weather[station_id]['Date/Time (LST)'] = pd.to_datetime(dict_weather[station_id]['Date/Time (LST)'])
#     print(station_id)
#     print(len(weather_joined_files))
#     break
# dict_weather

In [26]:
dict_weather[706]

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-121.76,49.24,AGASSIZ RCS,1100119,2021-01-01 00:00:00,2021,1,1,00:00,4.7,...,NaN,NaN,NaN,101.58,NaN,NaN,NaN,NaN,NaN,NaN
1,-121.76,49.24,AGASSIZ RCS,1100119,2021-01-01 01:00:00,2021,1,1,01:00,4.7,...,NaN,NaN,NaN,101.49,NaN,NaN,NaN,NaN,NaN,NaN
2,-121.76,49.24,AGASSIZ RCS,1100119,2021-01-01 02:00:00,2021,1,1,02:00,4.4,...,NaN,NaN,NaN,101.41,NaN,NaN,NaN,NaN,NaN,NaN
3,-121.76,49.24,AGASSIZ RCS,1100119,2021-01-01 03:00:00,2021,1,1,03:00,4.5,...,NaN,NaN,NaN,101.30,NaN,NaN,NaN,NaN,NaN,NaN
4,-121.76,49.24,AGASSIZ RCS,1100119,2021-01-01 04:00:00,2021,1,1,04:00,4.7,...,NaN,NaN,NaN,101.20,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,-121.76,49.24,AGASSIZ RCS,1100119,2022-01-31 19:00:00,2022,1,31,19:00,3.2,...,NaN,NaN,NaN,102.36,NaN,NaN,NaN,NaN,NaN,NaN
26300,-121.76,49.24,AGASSIZ RCS,1100119,2022-01-31 20:00:00,2022,1,31,20:00,3.6,...,NaN,NaN,NaN,102.37,NaN,NaN,NaN,NaN,NaN,NaN
26301,-121.76,49.24,AGASSIZ RCS,1100119,2022-01-31 21:00:00,2022,1,31,21:00,3.4,...,NaN,NaN,NaN,102.38,NaN,NaN,NaN,NaN,NaN,NaN
26302,-121.76,49.24,AGASSIZ RCS,1100119,2022-01-31 22:00:00,2022,1,31,22:00,3.3,...,NaN,NaN,NaN,102.34,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
dict_weather[706].columns

Index(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID',
       'Date/Time (LST)', 'Year', 'Month', 'Day', 'Time (LST)', 'Temp (°C)',
       'Temp Flag', 'Dew Point Temp (°C)', 'Dew Point Temp Flag',
       'Rel Hum (%)', 'Rel Hum Flag', 'Precip. Amount (mm)',
       'Precip. Amount Flag', 'Wind Dir (10s deg)', 'Wind Dir Flag',
       'Wind Spd (km/h)', 'Wind Spd Flag', 'Visibility (km)',
       'Visibility Flag', 'Stn Press (kPa)', 'Stn Press Flag', 'Hmdx',
       'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather'],
      dtype='object')

## Add Weather Data to Traffic Time Data

In [28]:
def find_weather_info(tt_entry, dict_tt_help_idx):
    station_id = dict_matched_ids[tt_entry[dict_tt_help_idx['AssetNumber']]] 
    weather_data = dict_weather[station_id]

    weather_record = weather_data[(weather_data['Date/Time (LST)'].dt.date == tt_entry[dict_tt_help_idx['CreatedDate']].date())
                                  & (weather_data['Date/Time (LST)'].dt.hour == tt_entry[dict_tt_help_idx['CreatedDate']].hour)]

    return weather_record

In [29]:
def add_weather_data(df_tt, start, stop):
    print(start, stop)
    df_tt = df_tt.reset_index()

    df_tt['Weather:Temp'] = ''
    df_tt['Weather:Precip'] = ''
    df_tt['Weather:WindSpd'] = ''

    dict_tt_help_idx = {'Weather:Temp': df_tt.columns.get_loc('Weather:Temp'),
                        'Weather:Precip': df_tt.columns.get_loc('Weather:Precip'),
                        'Weather:WindSpd': df_tt.columns.get_loc('Weather:WindSpd'),
                        'CreatedDate': df_tt.columns.get_loc('CreatedDate'),
                        'AssetNumber': df_tt.columns.get_loc('AssetNumber')}
    
    df_tt_workzone = np_funct(df_tt, dict_tt_help_idx, start, stop)
    df_tt_workzone.to_csv(os.path.join(output_root_path, f'processed_travel_time_{start}-{stop}.csv'))  
    return df_tt_workzone

In [30]:
def np_funct(df_tt, dict_tt_help_idx, start, stop):
    np_tt_workzone = df_tt.to_numpy()

    for tt_idx, tt_entry in enumerate(np_tt_workzone[start:stop], start):
#         if tt_entry[dict_tt_help_idx['AssetNumber']] in dict_matched_ids:
        weather_record = find_weather_info(tt_entry, dict_tt_help_idx)

        df_tt.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
        df_tt.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
        df_tt.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

        if tt_idx % 10000 == 0:
                print(f'{(tt_idx+1)/len(np_tt_workzone)*100:.2f}%')

    return df_tt

In [31]:
def dict_funct():
    temp = df_tt_workzone.to_dict('records')
    tt_idx = 0

    for tt_entry in temp:
        if tt_entry['AssetNumber'] in dict_matched_ids:
            weather_record = find_weather_info(tt_entry)

            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

        if tt_idx % 10000 == 0:
            print(f'{(tt_idx+1)/len(df_tt_workzone)*100:.2f}%')
        tt_idx += 1

    return df_tt_workzone

In [32]:
def normal_funct():
    # df_valid['Weather:Temp'] =  df_valid.apply(add_weather, axis=1)

    for tt_idx, tt_entry in df_tt_workzone.iterrows():
        if tt_idx % 10000 == 0:
            print(f'{(tt_idx+1)/len(df_tt_workzone)*100:.2f}%')

        if tt_entry['AssetNumber'] in dict_matched_ids:
            weather_record = find_weather_info(tt_entry)
            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
            df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

    return df_tt_workzone

In [33]:
%%time
df_tt_workzone = add_weather_data(df_tt_workzone, start=0, stop=len(df_tt_workzone)//2)

0 683869
0.00%
0.73%
1.46%
2.19%
2.92%
3.66%
4.39%
5.12%
5.85%
6.58%
7.31%
8.04%
8.77%
9.50%
10.24%
10.97%
11.70%
12.43%
13.16%
13.89%
14.62%
15.35%
16.09%
16.82%
17.55%
18.28%
19.01%
19.74%
20.47%
21.20%
21.93%
22.67%
23.40%
24.13%
24.86%
25.59%
26.32%
27.05%
27.78%
28.51%
29.25%
29.98%
30.71%
31.44%
32.17%
32.90%
33.63%
34.36%
35.09%
35.83%
36.56%
37.29%
38.02%
38.75%
39.48%
40.21%
40.94%
41.67%
42.41%
43.14%
43.87%
44.60%
45.33%
46.06%
46.79%
47.52%
48.25%
48.99%
49.72%
CPU times: user 8h 52min 29s, sys: 9.61 s, total: 8h 52min 38s
Wall time: 8h 52min 53s


In [34]:
# %%time
# df_tt_workzone = add_weather_data(df_tt_workzone, start=len(df_tt_workzone)//2, stop=len(df_tt_workzone))

# Get Counters Data


In [35]:
from dateutil.parser import parse

In [36]:
df_matched_counters_ids = pd.read_excel(os.path.join(counter_path, 'NearestCountStationsToAssets.xlsx'))[['Asset_Numb', 'prefixSiteNo']]
dict_matched_counters_ids = df_matched_counters_ids.set_index('Asset_Numb')['prefixSiteNo'].to_dict()
df_matched_counters_ids

,Asset_Numb,prefixSiteNo
0,23023696729,17-044EW
1,98128214373,17-044EW
2,25033704574,23-009NS
3,71808680042,23-010NS
4,27743462715,23-010NS
...,...,...
619,12066281555,21-011NS
620,62134790681,23-010NS
621,33379829945,23-010NS
622,21272146931,P-27-1EW


In [37]:
dict_traffic = {}
ignore_files = [ '21-012NS', '23-005EW']

# for site_no in df_matched_counters_ids['prefixSiteNo'].unique():
for site_no in os.listdir(os.path.join(counter_path, f'TrafficInfo')):
    if site_no in ignore_files:
        continue
        
    counter_joined_files = glob.glob(os.path.join(counter_path, f'TrafficInfo/{site_no}/*.xls'))
    
    if site_no == '23-009NS':
        counter_joined_files.remove(os.path.join(counter_path, 'TrafficInfo/23-009NS/DC11 - Site Wire Cache RA 23-009NS - NY on 08-22-2011_0.xls'))
        
    for f in counter_joined_files:
        df_traffic = pd.read_excel(f)
        
    dict_traffic[site_no] = {}   
    dict_traffic[site_no]['AADT'] = df_traffic.iloc[-1][13]
    dict_traffic[site_no]['CreatedDate'] = pd.to_datetime(df_traffic.iloc[-1][1])
    
    print(site_no, df_traffic.shape)
    
dict_traffic

27-001E (49, 24)
21-937NS (49, 24)
27-022E (49, 24)
P-17-1EW (824, 34)
P-27-1EW (822, 30)
27-0404 (49, 24)
27-001W (49, 24)
21-011NS (98, 24)
21-013EW (98, 24)
23-001NS (49, 24)
27-003EW (49, 24)
21-006NS (98, 24)
17-044EW (98, 24)
17-015E (99, 24)
27-036N (49, 24)
P-17-8NS (522, 35)
P-23-2EW (824, 32)
P-23-1EW (824, 32)
P-23-3NS (824, 32)
23-010NS (49, 24)
27-027S (49, 24)
23-009NS (49, 24)
27-046NS (49, 24)
27-009NS (49, 24)
21-002NS (98, 24)
27-0401 (49, 24)
17-015W (98, 24)
27-033E (49, 24)
W-23-2EW (81, 19)
27-022W (49, 24)
27-036S (49, 24)
27-030NS (49, 24)
27-033W (49, 24)


{'27-001E': {'AADT': 9487.004,
  'CreatedDate': Timestamp('2014-01-20 16:37:11')},
 '21-937NS': {'AADT': 3767, 'CreatedDate': Timestamp('2007-02-14 18:13:50')},
 '27-022E': {'AADT': 7449.4580000000005,
  'CreatedDate': Timestamp('2014-01-20 16:38:39')},
 'P-17-1EW': {'AADT': nan, 'CreatedDate': NaT},
 'P-27-1EW': {'AADT': nan, 'CreatedDate': NaT},
 '27-0404': {'AADT': 3232.165,
  'CreatedDate': Timestamp('2014-01-20 16:37:15')},
 '27-001W': {'AADT': 9435.631000000001,
  'CreatedDate': Timestamp('2014-01-20 16:37:12')},
 '21-011NS': {'AADT': 1352.8570000000002,
  'CreatedDate': Timestamp('2018-01-25 14:14:59')},
 '21-013EW': {'AADT': 402.704,
  'CreatedDate': Timestamp('2018-01-25 14:15:08')},
 '23-001NS': {'AADT': 1542.2810000000002,
  'CreatedDate': Timestamp('2018-01-25 14:13:14')},
 '27-003EW': {'AADT': 3608.106,
  'CreatedDate': Timestamp('2014-01-20 16:39:29')},
 '21-006NS': {'AADT': 3140.103,
  'CreatedDate': Timestamp('2017-02-09 13:15:23')},
 '17-044EW': {'AADT': 1948.797000000

In [38]:
# parse(df_traffic.iloc[1][0], fuzzy=True).year 

## Add Counters Data to Traffic Time Data

## Export Potential Wrong Data

In [39]:
def find_delay_diff():
    diff_delay = df_valid[df_valid['DelayTime'] != df_valid['CalculatedDelayTime']]
    diff_delay.to_csv(os.path.join(output_root_path, 'wrong_DelayTime.csv'), header = True, index=True)
    return diff_delay

In [40]:
def find_state_diff():
    diff_state = df_valid[df_valid['State'] != df_valid['CalculatedState']]
    diff_state.to_csv(os.path.join(output_root_path, 'wrong_State.csv'), header = True, index=True)
    return diff_state 

# Exploratory Data Analysis

## General Info about dataframe

In [41]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3809841 entries, 0 to 3809840
Data columns (total 18 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   AllTasksIndex          int64         
 1   AssetNumber            int64         
 2   AssetIdentifier        object        
 3   CreatedDate            datetime64[ns]
 4   UnderConstructionTime  float64       
 5   BaselineTime           float64       
 6   DelayTime              float64       
 7   State                  object        
 8   SegmentLength          float64       
 9   CalculatedDelayTime    float64       
 10  CreatedDate:Weekday    int64         
 11  CreatedDate:Year       int64         
 12  CreatedDate:Month      int64         
 13  CreatedDate:Hour       int64         
 14  CreatedDate:Season     int64         
 15  CreatedDate:DayType    int64         
 16  CreatedDate:HourType   int64         
 17  DelayLevel             int64         
dtypes: datetime64[ns](1), 

In [42]:
df_valid.isnull().sum()

AllTasksIndex              0
AssetNumber                0
AssetIdentifier            0
CreatedDate                0
UnderConstructionTime      0
BaselineTime               0
DelayTime                  0
State                    178
SegmentLength              0
CalculatedDelayTime        0
CreatedDate:Weekday        0
CreatedDate:Year           0
CreatedDate:Month          0
CreatedDate:Hour           0
CreatedDate:Season         0
CreatedDate:DayType        0
CreatedDate:HourType       0
DelayLevel                 0
dtype: int64

In [43]:
print(max(df_valid['CreatedDate:Month']))
print(min(df_valid['CreatedDate:Month']))

12
1


In [44]:
print(max(df_valid['CreatedDate:Year']))
print(min(df_valid['CreatedDate:Year']))

2021
2020


## Plot

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt
# from PIL import Image

%matplotlib inline
sns.set_style("darkgrid")
plt.style.use('ggplot')

STEP_SIZE = 0.5


In [46]:
def draw_time_plot(loc, loc_id, time_type, plt_name):
    fig, ax = plt.subplots()

    # loc.plot.scatter(x='CreatedDate:DayName', y=['UnderConstructionTime', 'BaselineTime'])
    # loc.plot.scatter(x='CreatedDate:DayName', y='UnderConstructionTime')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['UnderConstructionTime'], color='purple', marker='o', label='UnderConstructionTime (After)')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['BaselineTime'], color='orange', marker='h', label='BaselineTime (Before)')

    x, y  = [], []
    
    if time_type == 'Hour':
        for hour in range(24):
            loc_hour = loc.loc[loc[f'CreatedDate:{time_type}'] == hour]
            x.append(hour)
            y.append(loc_hour['UnderConstructionTime'].mean())
    
    elif time_type == 'DayName':
         for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            loc_day = loc.loc[loc[f'CreatedDate:{time_type}'] == day]
            x.append(day)
            y.append(loc_day['UnderConstructionTime'].mean())
    
    temp = loc['BaselineTime'].mean()
    plt.plot(x, y, color='purple', marker='o', label='UnderConstructionTime (After)')
    plt.plot(x, [temp for i in range(len(y))], color='orange', marker='h', label='BaselineTime (Before)')
    
#     start, end = ax.get_ylim()
#     plt.yticks(np.arange(np.round(start), np.round(end)+STEP_SIZE, step=STEP_SIZE))
    if time_type == 'Hour':
        plt.xticks(np.arange(0, 24, step=1))
    
    fig.set_size_inches(14, 10)
    ax.legend()
    ax.grid(True)
    plt.title(f'Travel Time for AssetID {loc_id} ({time_type})')
    plt.xlabel(time_type)
    plt.ylabel('TravelTime')
    plt.tight_layout()
    plt.savefig(f'{output_root_path}/{plt_name}.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


### Draw plot for selected asset ids

In [47]:
locations_id = [80002045894, 50253718586, 44853071770, 12738620179, 37528363723, 43805438913, 68881926366, 36674989230, 34592545857, 22685608926]


In [48]:
# for location_id in locations_id:
#     for time_type in ['DayName', 'Hour']:
#         location = df_valid.loc[df_valid['AssetNumber'] == location_id]
#         draw_time_plot(location, location_id, time_type, plt_name=f'{location_id}_{time_type}')


In [49]:
# !zip plots.zip ../working/*.png